# Inport Necessary Libralies

In [14]:
import praw
import configparser
import pandas as pd
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
import seaborn as sns

# Set up configuration

In [15]:
# Set up search conditions 

#Define the broad search term (replaces the 'search_phrases' list)
SEARCH_TERM = '(Cybertruck Problems OR Cybertruck Issues OR Cybertruck Rust)'

# Set up search conditions (re-stating for clarity, assuming they are defined earlier)
SUBREDDITS = ['popular', 'all', 'technology', 'worldnews']
POST_LIMIT = 1000

# Read config.ini
config = configparser.ConfigParser()
config.read('config.ini')

# Initialise Praw Client
try:
    # user authentication
    reddit = praw.Reddit(
        client_id=config['reddit']['client_id'],
        client_secret=config['reddit']['client_secret'],
        user_agent=config['reddit']['user_agent'],
        username=config['reddit']['username'],
        password=config['reddit']['password']
    )
    # Ensure the log in successful.
    print(f"PRAW client initialized. Logged in as: {reddit.user.me().name}")

except KeyError as e:
    print(f"FATAL ERROR: Missing Reddit credential in config.ini. Check key: {e}")
    exit()
except Exception as e:
    print(f"An error occurred during PRAW initialization or login: {e}")
    exit()

PRAW client initialized. Logged in as: Afraid-Medicine4769


# Retrive and Collect Submissions

In [16]:
# Define the flexible search terms for Python to check
search_phrases = ['cybertruck', 'tesla truck', 'musk truck', 'stainless steel rust']

collected_posts = []

print(f"\n--- Starting data collection across {len(SUBREDDITS)} subreddits ---")

for subreddit_name in SUBREDDITS:
    print(f"Searching r/{subreddit_name}...")
    try:
        subreddit = reddit.subreddit(subreddit_name)
        
        # Use .top() - Guaranteed to retrieve 1000 posts from popular/all
        for submission in subreddit.new(limit=POST_LIMIT):
            
            # Combine the title and text to search for the term, and convert to lowercase
            full_content = (submission.title + ' ' + submission.selftext).lower()
            
            # Filter the posts yourself in Python for any of the flexible phrases
            if any(phrase in full_content for phrase in search_phrases):
                collected_posts.append({
                    'id': submission.id,
                    'subreddit': subreddit_name,
                    'title': submission.title,
                    'text': submission.selftext,
                    'score': submission.score,
                    'num_comments': submission.num_comments,
                    'created_utc': submission.created_utc,
                    'url': submission.url
                })
            
    except Exception as e:
        print(f"Skipping r/{subreddit_name} due to an error: {e}")
        continue # Move to the next subreddit

print(f"\nData Collection Complete. Total submissions collected: {len(collected_posts)}")


--- Starting data collection across 4 subreddits ---
Searching r/popular...
Searching r/all...
Searching r/technology...
Searching r/worldnews...

Data Collection Complete. Total submissions collected: 3
